In [8]:
import numpy as np
import pandas as pd
import h5py
import os
import glob
from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt
import matplotlib.dates as md
import warnings
from scipy.interpolate import interp2d
from scipy.interpolate import interp1d
from scipy.interpolate import make_interp_spline
warnings.filterwarnings('ignore')
import scipy.ndimage as ndimage
from matplotlib import cm
from matplotlib.colors import LogNorm
import matplotlib.ticker as ticker
from sklearn import preprocessing
import seaborn as sns
from scipy.stats import pearsonr
from scipy import stats

In [9]:
def simple_read(path):
    with open(path) as f:
        first_line = f.readline()
        header_line = int(first_line[0:-2].split(",")[0])-1
        #all_lines = f.readlines()
    data = pd.read_csv(path, sep=',', skiprows=header_line)
    data.columns = [i.strip() for i in data.columns]
    #date = all_lines[5][0:10].split(',')
    #dt = datetime(int(date[0]), int(date[1]), int(date[2]))

    acc = 0
    boo = False
    for letter in path:
        if letter == '2':
            boo = True
        elif boo and letter == '0':
            acc -= 1
            break
        acc += 1
    #print(path[:acc+4], path[acc+4:acc+6], path[acc+6:acc+8])
    dt = datetime(int(path[acc:acc+4]), int(path[acc+4:acc+6]), int(path[acc+6:acc+8]))
    
    for column in data.keys():
        if 'Time' in column:
            data[column] = dt + pd.to_timedelta(data[column], unit='seconds')
    return data.replace(-999999, np.NaN)

In [7]:
DAY2 = simple_read('C:/Users/taiwoajayi/Documents/Ozone/activate-mrg1_hu25_20220602_R0_L1_20230701T094923.ict')


start_index = np.where(DAY2["Time_Start"] == pd.to_datetime("2022-06-02 13:48:00"))
stop_index = np.where(DAY2["Time_Start"] == pd.to_datetime("2022-06-02 14:10:00"))

DAY2_subset = DAY2[start_index[0][0]:stop_index[0][0]]
DAY2_subset['CO_ppm'] = DAY2_subset['CO_ppm_DISKIN']*1000
DAY2_subset


KeyError: 'CO_ppm_DISKIN'

In [4]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY2_subset.GPS_Altitude_THORNHILL.min(), DAY2_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY2_subset['Bins'] = pd.cut(DAY2_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY2_subset_1000 = DAY2_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY2_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY2_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_1000 = DAY2_subset_1000[start_index:stop_index]
DAY1000_subset_1000


,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,CH4_ppm_DISKIN,CO_ppm_DISKIN,CO2_ppm_DISKIN,O3_ppbv_DISKIN,LWC_FCDP_VOIGT,Sc550_submicron_ZIEMBA,RH_Sc_submicron_ZIEMBA,Ext532_submicron_amb_ZIEMBA,fRH550_RH20to80_ZIEMBA,AEscatAMB_450to700nm_ZIEMBA,AEabsDRY_470to660nm_ZIEMBA,CO_ppm
count,31,31,31.000000,31.000000,31.000000,0.0,0.0,0.0,31.000000,3.100000e+01,31.000000,31.000000,31.000000,31.000000,31.000000,0.0,0.0
mean,2022-06-02 13:57:53.764954880,2022-06-02 13:57:54.764954880,32.257685,-65.149491,536.450592,NaN,NaN,NaN,19.645136,1.023138e-07,10.256219,50.438010,15.869404,1.646559,1.523214,NaN,NaN
min,2022-06-02 13:54:46.487500032,2022-06-02 13:54:47.487500032,32.208589,-65.305158,121.847785,NaN,NaN,NaN,16.090909,1.915370e-08,7.238000,46.200000,11.256667,1.254286,1.370600,NaN,NaN
25%,2022-06-02 13:57:48.635714304,2022-06-02 13:57:49.635714304,32.244939,-65.204104,326.578179,NaN,NaN,NaN,18.545259,3.899339e-08,9.176169,48.473901,15.111250,1.489907,1.433429,NaN,NaN
50%,2022-06-02 13:58:08,2022-06-02 13:58:09.000000256,32.259830,-65.163330,538.625517,NaN,NaN,NaN,19.632000,9.027000e-08,10.619545,50.655172,16.042500,1.615886,1.517143,NaN,NaN
75%,2022-06-02 13:58:29.758620672,2022-06-02 13:58:30.758620672,32.276021,-65.062549,743.969231,NaN,NaN,NaN,20.380000,1.289598e-07,11.479875,52.906429,17.459041,1.770250,1.552857,NaN,NaN
max,2022-06-02 13:59:27.500000,2022-06-02 13:59:28.500000,32.293133,-65.035237,951.478333,NaN,NaN,NaN,23.233333,3.230668e-07,12.707143,54.120253,19.130455,2.263750,1.797143,NaN,NaN
std,NaN,NaN,0.023269,0.080663,252.206520,NaN,NaN,NaN,1.470277,7.944510e-08,1.545906,2.554601,2.274934,0.227399,0.102404,NaN,NaN


In [5]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(DAY2_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY2_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_3000 = DAY2_subset_1000[start_index:stop_index]
DAY3000_subset_3000

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,CH4_ppm_DISKIN,CO_ppm_DISKIN,CO2_ppm_DISKIN,O3_ppbv_DISKIN,LWC_FCDP_VOIGT,Sc550_submicron_ZIEMBA,RH_Sc_submicron_ZIEMBA,Ext532_submicron_amb_ZIEMBA,fRH550_RH20to80_ZIEMBA,AEscatAMB_450to700nm_ZIEMBA,AEabsDRY_470to660nm_ZIEMBA,CO_ppm
count,67,67,67.000000,67.000000,67.000000,0.0,0.0,0.0,67.000000,6.700000e+01,67.000000,67.000000,58.000000,61.000000,63.000000,0.0,0.0
mean,2022-06-02 14:03:43.209282816,2022-06-02 14:03:44.209282560,32.269022,-65.105769,1922.245111,NaN,NaN,NaN,40.453837,1.247840e-08,5.371518,40.052697,7.215837,1.279915,0.865004,NaN,NaN
min,2022-06-02 13:50:07.969465600,2022-06-02 13:50:08.969465600,32.218661,-65.750983,1005.609375,NaN,NaN,NaN,20.133333,0.000000e+00,2.205000,32.860000,-7.240000,0.740000,0.214091,NaN,NaN
25%,2022-06-02 14:00:10,2022-06-02 14:00:11,32.256105,-65.180879,1463.450208,NaN,NaN,NaN,29.918750,0.000000e+00,4.456182,37.495000,6.022281,1.050000,0.671786,NaN,NaN
50%,2022-06-02 14:01:32.636363776,2022-06-02 14:01:33.636363776,32.270983,-65.073541,1913.302121,NaN,NaN,NaN,38.338931,5.441111e-09,5.434286,40.391667,7.566875,1.226429,0.893889,NaN,NaN
75%,2022-06-02 14:07:55.500000,2022-06-02 14:07:56.500000,32.281553,-64.994318,2381.528500,NaN,NaN,NaN,51.183333,1.381850e-08,6.194167,42.684375,8.949479,1.503750,1.109509,NaN,NaN
max,2022-06-02 14:09:52,2022-06-02 14:09:53,32.365645,-64.955685,2837.362000,NaN,NaN,NaN,68.190909,1.441531e-07,8.546250,46.755000,14.205000,2.007143,1.570625,NaN,NaN
std,NaN,NaN,0.028633,0.147561,540.263288,NaN,NaN,NaN,13.462948,2.435765e-08,1.367669,3.720053,3.451932,0.298861,0.329298,NaN,NaN


In [6]:

# Concatenate the dataframes vertically (along rows)
ocean = pd.concat([DAY1000_subset_1000, DAY1000_subset_1000b], axis=0)

# Reset the index of the merged dataframe
ocean.reset_index(drop=True, inplace=True)
ocean.describe()

NameError: name 'DAY1000_subset_1000b' is not defined

In [ ]:
# Concatenate the dataframes vertically (along rows)
ocean2 = pd.concat([DAY3000_subset_3000, DAY3000_subset_3000b], axis=0)

# Reset the index of the merged dataframe
ocean2.reset_index(drop=True, inplace=True)
ocean2.describe()

In [ ]:
# Concatenate the dataframes vertically (along rows)
North_America = pd.concat([DAY1000_subset_7, DAY1000_subset_7b, DAY1000_subset_17], axis=0)

# Reset the index of the merged dataframe
North_America.reset_index(drop=True, inplace=True)
North_America.describe()

In [ ]:
# Concatenate the dataframes vertically (along rows)
North_America2 = pd.concat([DAY7_subset_3000, DAY7b_subset_3000, DAY17_subset_3000], axis=0)

# Reset the index of the merged dataframe
North_America2.reset_index(drop=True, inplace=True)
North_America2.describe()

In [ ]:
# Concatenate the dataframes vertically (along rows)
Caribbean = pd.concat([DAY1000_subset_3, DAY1000_subset_8, DAY1000_subset_10, DAY1000_subset_10b, DAY1000_subset_11, DAY1000_subset_11b], axis=0)

# Reset the index of the merged dataframe
Caribbean.reset_index(drop=True, inplace=True)
Caribbean.describe()

In [ ]:
# Concatenate the dataframes vertically (along rows)
Caribbean2 = pd.concat([DAY3_subset_3000, DAY8_subset_3000, DAY10_subset_3000, DAY10b_subset_3000, DAY11_subset_3000, DAY11b_subset_3000, DAY13_subset_3000, DAY1000_subset_13b], axis=0)

# Reset the index of the merged dataframe
Caribbean2.reset_index(drop=True, inplace=True)
Caribbean2.describe()

In [ ]:
Africa = pd.concat([DAY1000_subset_13, DAY1000_subset_13b, DAY1000_subset_14])
# Reset the index of the merged dataframe
Africa.reset_index(drop=True, inplace=True)
Africa.describe()

In [ ]:
# Concatenate the dataframes vertically (along rows)
merged_df = pd.concat([Caribbean.describe(), Caribbean2.describe(), North_America.describe(), North_America2.describe(), ocean.describe(), ocean2.describe(), Africa.describe(), DAY14_subset_3000.describe()], axis=0)

# Reset the index of the merged dataframe
merged_df.reset_index(drop=False, inplace=True)

# Display the merged dataframe
merged_df
merged_df.to_csv('C:/Users/taiwoajayi/OneDrive - University of Arizona/python/co_mass_description.csv')